In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import Trainer, TrainingArguments, TrainerCallback
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import tensorflow as tf
import torch
import os
from datasets import Dataset
import time

In [3]:
# Configurar dispositivo como variable global
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
def check_gpu_memory():
    allocated_memory = torch.cuda.memory_allocated()
    cached_memory = torch.cuda.memory_reserved()
    print(f"Memoria GPU asignada: {allocated_memory / 1024 ** 2:.2f} MB")
    print(f"Memoria GPU reservada: {cached_memory / 1024 ** 2:.2f} MB")
    print(f"Memoria libre: {torch.cuda.get_device_properties(0).total_memory / 1024 ** 2 - allocated_memory / 1024 ** 2:.2f} MB")

check_gpu_memory()  # Verifica antes de entrenar

Memoria GPU asignada: 0.00 MB
Memoria GPU reservada: 0.00 MB
Memoria libre: 4095.56 MB


In [5]:

def clear_gpu_memory():
    # Comando para restablecer la GPU usando nvidia-smi
    os.system('nvidia-smi --gpu-reset -i 0')

    # Limpiar la caché de memoria de CUDA
    torch.cuda.empty_cache()

    # También puedes intentar vaciar la caché de la memoria de TensorFlow, si lo estás usando
    try:
        import tensorflow as tf
        tf.keras.backend.clear_session()
        print("Sesión de TensorFlow limpiada.")
    except ImportError:
        pass

    # Verificación de memoria
    print("Verificando el estado de la memoria de la GPU después de limpiar...")
    time.sleep(2)  # Un pequeño retraso para asegurar que los cambios se apliquen
    os.system('nvidia-smi')

# Llamada a la función para limpiar la memoria de la GPU
clear_gpu_memory()


Sesión de TensorFlow limpiada.
Verificando el estado de la memoria de la GPU después de limpiar...


In [10]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()  # Carga variables del archivo .env
token = os.getenv("HF_TOKEN")
login(token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [11]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Asignar el token de padding al token de "fin de oración" (eos_token)
tokenizer.pad_token = tokenizer.eos_token

tokenizer.model_max_length = 1024  # Asegura que el tokenizador usa el límite correcto
model.to(device)  # Mover el modelo a la GPU si está disponible

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)